In [18]:
#Importacoes necessarias
import csv
import pandas as pd
import numpy as np
import nltk
import re
import collections
import math
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('rslp')
import heapq
import time
import bisect

#Leitura do Arquivo CSV
results = pd.read_csv('https://raw.githubusercontent.com/BielMaracaja/RI/master/ri_lab_06/results.csv')
#results = results.astype(str)
documents = results['text']

json = pd.read_json('https://raw.githubusercontent.com/BielMaracaja/RI/master/results_final.json')
feedback = {json['query'][i]:json['docs'][i] for i in range(10)}

tokenizador = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')

stopwords = nltk.corpus.stopwords.words('portuguese') 

indices = {}

T = results.text.count()

for i in range(len(results)):
  text = results.text[i]
  words = [word for word in tokenizador.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  
  for t in words:
    if t not in indices.keys():
      indices[t] = []
    indices[t].append(i)    
for elem in indices.items():
  d = dict(collections.Counter(elem[1]))
  indices[elem[0]] = list(d.items())  
for word in indices:
  k = len(indices[word])
  IDF = round(np.log((T+1)/k),2)
  indices[word].append(IDF)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


#1
**Escolha um documento dentre aqueles da base do aluno Bernardi e crie uma consulta que você acha que tem boas chances de recuperar este documento. Em seguida, avalie os resultados de tal consulta usando a métrica de avaliação Reciprocal Rank**

In [19]:
ndoc = 202
document = results.loc[ndoc]
query = 'bolsa família'

document.title

'Bolsonaro e o desenvolvimento dos filhos do Bolsa Família: o que os pesquisadores realmente dizem'

Pegaremos as definições dos modelos vetoriais do Lab passado.

In [0]:
def vsm_binario(query, document):
  pontuacao = 0
  
  query_tokens = query.split()
  
  tokens_doc = document.split()
  
  for token in query_tokens:
    pontuacao += (token in tokens_doc)
  
  return pontuacao

In [0]:
def vsm_tf(query, document):
  pontuacao = 0
  
  tokens_doc = document.split()
  
  query_tokens = query.split()
  
  for palavra in query_tokens:
    pontuacao += tokens_doc.count(palavra)
  
  return pontuacao

In [0]:
def vsm_tfidf(query, document):
  score = 0
  
  doc_tokens = document.split()
  
  query_tokens = query.split()
  
  for word in query_tokens:
    
    cwd = doc_tokens.count(word)
    
    if word in indices:
      score += cwd * indices[word][-1]
  
  return round(score,2)

In [0]:
def vsm_bm25(query, document, k):
  pontuacao = 0
  
  tokens_doc = document.split()
  
  query_tokens = query.split()
  
  palavras = [palavra for palavra in query_tokens if palavra in tokens_doc]
    
  for palavra in palavras:
    cwd = tokens_doc.count(palavra)
    dfw = 0
    
    if palavra in indices:
      dfw = len(indices[palavra][:-1])
      pontuacao += (((k+1) * cwd) / (cwd + k)) * np.log10(((T+1) / dfw)) if dfw != 0 else 0
  return round(pontuacao,2)

Portanto, criaremos as tabelas com os top-k documentos de cada modelo.

In [0]:
def top_models_create(query,k):
  db = []
  
  dtf = []
  
  dtfidf = []
  
  dbm25 = []
  
  for i in range(len(results)):
    doc = results.text[i].lower()
    
    bisect.insort(db, (vsm_binario(query, doc), i))    
    bisect.insort(dtf, (vsm_tf(query,doc), i))    
    bisect.insort(dtfidf, (vsm_tfidf(query,doc), i))    
    bisect.insort(dbm25, (vsm_bm25(query,doc,20), i))
  
  db.reverse()
  
  dtf.reverse()
  
  dtfidf.reverse()
  
  dbm25.reverse()
  
  return db[:k], dtf[:k], dtfidf[:k], dbm25[:k]

**Buscando Resolução dos modelos**

In [25]:
binario_top, tf_top, tfidf_top, bm_25_top = top_models_create(query,10)

query_df = pd.DataFrame()

query_df['Binario'] = binario_top
query_df['TF'] = tf_top
query_df['TF-IDF'] = tfidf_top
query_df['BM25'] = bm_25_top

query_df.index+=1
query_df

,Binario,TF,TF-IDF,BM25
1,"(2, 236)","(17, 202)","(35.04, 202)","(11.15, 202)"
2,"(2, 235)","(9, 248)","(14.31, 248)","(4.5, 248)"
3,"(2, 215)","(8, 206)","(12.72, 206)","(4.14, 206)"
4,"(2, 202)","(7, 5)","(11.13, 5)","(3.94, 236)"
5,"(2, 68)","(6, 150)","(9.73, 236)","(3.76, 5)"
6,"(2, 25)","(5, 236)","(9.54, 150)","(3.35, 150)"
7,"(1, 248)","(3, 225)","(7.44, 214)","(2.95, 214)"
8,"(1, 242)","(3, 214)","(7.44, 141)","(2.95, 141)"
9,"(1, 239)","(3, 141)","(7.44, 137)","(2.95, 137)"
10,"(1, 225)","(3, 137)","(4.96, 133)","(2.05, 133)"


In [26]:
#Classificacao Recíproca

def clas_reciproco(tuples, docId):
  n = 1.0;
  
  for r,doc in tuples:
    
    if doc == docId:
      return  [round(1 / n, 2)]
    
    else:
      n += 1

clasf_df = pd.DataFrame()

clasf_df['Binario'] = clas_reciproco(query_df['Binario'], ndoc)

clasf_df['TF'] = clas_reciproco(query_df['TF'], ndoc)

clasf_df['TF-IDF'] = clas_reciproco(query_df['TF-IDF'], ndoc)

clasf_df['BM25'] = clas_reciproco(query_df['BM25'], ndoc)

clasf_df.index+=1

clasf_df

,Binario,TF,TF-IDF,BM25
1,0.25,1.0,1.0,1.0


#2
**A partir do gabarito fornecido em OBS1, calcule o MAP para cada algoritmo abaixo e aponte qual obteve o melhor resultado. Para os cálculos do MAP, considere que um documento é relevante para uma dada consulta se este documento estiver entre os documentos do gabarito para essa consulta, senão ele deve ser considerado irrelevante.** 

In [0]:
def indices_doc(model):
  return [doc for score,doc in model]

def intersecao(a,b):
  return [elem for elem in a if elem in b]

def calcula_AP(query):
  relevant_docs = []

  for doc_info in feedback[query]:
    
    row = results.loc[results.url == doc_info['URL']]
    
    relevant_docs.append(row.index[0])
  
  binary, tf, tfidf, bm25 = top_models_create(query, 5)
  
  binary = indices_doc(binary)
  
  tf = indices_doc(tf)
  
  tfidf = indices_doc(tfidf)
  
  bm25 = indices_doc(bm25)
  
  ap_binary = len(intersecao(binary, relevant_docs)) / len(binary)
  
  ap_tf = len(intersecao(tf, relevant_docs)) / len(tf)
  
  ap_tfidf = len(intersecao(tfidf, relevant_docs)) / len(tfidf)
  
  ap_bm25 = len(intersecao(bm25, relevant_docs)) / len(bm25)
  
  return ap_binary, ap_tf, ap_tfidf, ap_bm25

**Calculo do Mean Average Precision**

In [0]:
def calcula_MAP(queries):
  sum_binary = 0
  
  sum_tf = 0
  
  sum_tfidf = 0
  
  sum_bm25 = 0
  
  for query in queries:
    
    ap_binary, ap_tf, ap_tfidf, ap_bm25 = calcula_AP(query)
    
    sum_binary += ap_binary
    
    sum_tf += ap_tf
    
    sum_tfidf += ap_tfidf
    
    sum_bm25 += ap_bm25
  
  
  map_binary = round(sum_binary / len(queries),2)
  
  map_tf = round(sum_tf / len(queries),2)
  
  map_tfidf = round(sum_tfidf / len(queries),2)
  
  map_bm25 = round(sum_bm25 / len(queries),2)
  
  return map_binary, map_tf, map_tfidf, map_bm25

In [29]:
map_binary, map_tf, map_tfidf, map_bm25 = calcula_MAP(feedback.keys())

clasf_df = pd.DataFrame()

clasf_df['Binario'] = [map_binary]

clasf_df['TF'] = [map_tf]

clasf_df['TF-IDF'] = [map_tfidf]

clasf_df['BM25'] = [map_bm25]

clasf_df.index+=1
clasf_df

,Binario,TF,TF-IDF,BM25
1,0.1,0.02,0.18,0.18


Conseguimos entender que por definicao o valor do (MAP) varia entre 0 e 1. Esses modelos que calculamos tem uma baixa precisao, na procura de documentos especificos.

#3
**Repita Q2 usando a avaliação multi-nível DCG. Utilize o campo "level" do gabarito para o cálculo do DCG e do idealDCG.**

Calculo DCG

In [0]:
def calculaDCG(modelo, niveis):
  
  dcg = 0.0
  
  for i in range(1,len(modelo)+1):
    
    doc = modelo[i-1]
    
    nivel = getNivel(doc, niveis)
    
    dcg += (2^nivel) / np.log2(i + 1.0)
    
  return dcg

Modelos DCG

In [0]:
def modelosDCG(consulta):
  
  documentos_relevantes = {}

  for info_documento in feedback[consulta]:
    coluna = results.loc[results.url == info_documento['URL']]
    documentos_relevantes[coluna.index[0]] = info_documento['level']
    
  binario, tf, tfidf, bm25 = top_models_create(consulta, 5)
  
  binario, tf, tfidf, bm25 = todosOsDocumentos(binario, tf, tfidf, bm25)
  
  dcg_binario = round(calculaDCG(binario, setNivel(binario, documentos_relevantes)),2)
  
  dcg_tf = round(calculaDCG(tf, setNivel(tf, documentos_relevantes)),2)
  
  dcg_tfidf = round(calculaDCG(tfidf, setNivel(tfidf, documentos_relevantes)),2)
  
  dcg_bm25 = round(calculaDCG(bm25, setNivel(bm25, documentos_relevantes)),2)
  
  return dcg_binario, dcg_tf, dcg_tfidf, dcg_bm25

In [0]:
def modelosDCGIdeal(consulta):
  
  documentos_relevantes = {}

  for info_documento in feedback[consulta]:
    coluna = results.loc[results.url == info_documento['URL']]
    documentos_relevantes[coluna.index[0]] = info_documento['level']
    
  binario, tf, tfidf, bm25 = top_models_create(consulta, 5)
  
  binario = indices_doc(binario)
  
  tf = indices_doc(tf)
  
  tfidf = indices_doc(tfidf)
  
  bm25 = indices_doc(bm25)
  
  niveis_binario, niveis_tf, niveis_tfidf, niveis_bm25 = todosOsNiveis(binario, tf, tfidf, bm25, documentos_relevantes)
  
  binario, tf, tfidf, bm25 = extraiDocumentos(niveis_binario, niveis_tf, niveis_tfidf, niveis_bm25)
  
  idcg_binario = round(calculaDCG(binario, niveis_binario), 2)
  
  idcg_tf = round(calculaDCG(tf, niveis_tf), 2)
  
  idcg_tfidf = round(calculaDCG(tfidf, niveis_tfidf), 2)
  
  idcg_bm25 = round(calculaDCG(bm25, niveis_bm25), 2)
  
  return idcg_binario, idcg_tf, idcg_tfidf, idcg_bm25

Definicao de algumas funcoes auxiliares

In [0]:
def getNivel(d, l):
  
  for level,doc in l:
    if doc == d:
      return level
    
def setNivel(m, d):
  
  modelo = [(0, documento) for documento in m if documento not in d]
  
  dic = [(v, k) for k, v in d.items()]
  
  res = modelo + dic
  
  res.sort(reverse=True)
  
  return res

def todosOsDocumentos(binario, tf, tfidf, bm25):
  return indices_doc(binario), indices_doc(tf), indices_doc(tfidf), indices_doc(bm25)
    
def todosOsNiveis(binario, tf, tfidf, bm25, rd):
  return setNivel(binario, rd), setNivel(tf, rd), setNivel(tfidf, rd), setNivel(bm25, rd)

def extraiDocumentos(binario, tf, tfidf, bm25):
  return [documento for nivel, documento in binario], [documento for nivel, documento in tf], [documento for nivel, documento in tfidf], [documento for nivel, documento in bm25]

Agora, defina-mos os valores do DCG de acordo com os modelos

In [34]:
resultados_consultas = {}

for consulta in feedback.keys():
  
  dcg_binario, dcg_tf, dcg_tfidf, dcg_bm25 = modelosDCG(consulta)
  
  idcg_binario, idcg_tf, idcg_tfidf, idcg_bm25 = modelosDCGIdeal(consulta)
  
  
  binario = (dcg_binario, idcg_binario)
  
  tf = (dcg_tf, idcg_tf)
  
  tfidf = (dcg_tfidf, idcg_tfidf)
  
  bm25 = (dcg_bm25, idcg_bm25)
  
  resultados = [binario, tf, tfidf, bm25]
  
  resultados_consultas[consulta] = resultados
  
  tabela_3 = pd.DataFrame()

tabela_3['Consulta'] = feedback.keys()

tabela_3['Binário'] = [resultados_consultas[consulta][0] for consulta in feedback.keys()]

tabela_3['TF'] = [resultados_consultas[consulta][1] for consulta in feedback.keys()]

tabela_3['TF-IDF'] = [resultados_consultas[consulta][2] for consulta in feedback.keys()]

tabela_3['BM25'] = [resultados_consultas[consulta][3] for consulta in feedback.keys()]

tabela_3.index += 1

tabela_3

,Consulta,Binário,TF,TF-IDF,BM25
1,território palestino,"(5.9, 15.65)","(5.9, 15.65)","(5.51, 14.98)","(5.51, 14.98)"
2,recessão mundial,"(11.58, 14.98)","(9.77, 14.98)","(9.77, 14.98)","(9.77, 14.98)"
3,ditadura militar,"(5.9, 17.17)","(5.9, 17.17)","(5.9, 17.17)","(5.9, 17.17)"
4,muro das lamentações,"(18.08, 19.29)","(5.9, 21.3)","(19.29, 19.29)","(19.29, 19.29)"
5,brasil e argentina,"(8.9, 17.5)","(5.9, 18.17)","(7.79, 17.5)","(7.79, 17.5)"
6,golpe militar,"(5.9, 20.67)","(5.9, 20.67)","(8.4, 20.04)","(8.4, 20.04)"
7,governo bolsonaro,"(5.9, 16.54)","(5.9, 16.54)","(5.9, 16.54)","(5.9, 16.54)"
8,ministro da economia,"(5.9, 17.17)","(5.9, 17.17)","(5.9, 17.17)","(5.9, 17.17)"
9,prisão de Temer,"(5.9, 13.43)","(5.9, 13.43)","(10.29, 12.05)","(10.29, 12.05)"
10,Congresso Nacional,"(5.9, 9.65)","(5.9, 9.65)","(5.9, 9.65)","(5.9, 9.65)"


Na tebela acima, podemos ver a disposicao dos resultados que foram encontrados para o DCG comum e o ideal, nas tuplas para os modelos usados (Binario, TF, TF-IDF e BM-25)